# RideScore DC — LTS from OSM (MVP)

This notebook builds a **Level of Traffic Stress (LTS)** layer for Washington, DC from OpenStreetMap tags and an explicit ruleset. Optionally, it joins DC crash points and computes a composite **RideScore (0–100)**. Outputs a map-ready **GeoJSON** and an inline Folium map with layer toggles.

**Method anchors:** Mekuria–Furth–Nixon LTS (1–4) and public agency adaptations (Montgomery, Fairfax). Thresholds here are **starting points**—we’ll tune with the group.  [oai_citation:0‡NACTO](https://nacto.org/wp-content/uploads/1005-low-stress-bicycling-network-connectivity.pdf?utm_source=chatgpt.com)

In [13]:
# 1) Install + imports

# If running locally (or in Kaggle with Internet on), uncomment:
!pip -q install osmnx geopandas shapely pyproj folium requests rtree

import os, json, math, re, warnings
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import osmnx as ox
import folium
from folium import Element
from folium.plugins import MeasureControl
import requests

warnings.filterwarnings("ignore")
ox.settings.use_cache = True
ox.settings.log_console = False

In [14]:
# 2) Config

PLACE_NAME = "Washington, District of Columbia, USA"
CRS = "EPSG:4326"
YEARS_BACK = 5                 # crash window
CRASH_ENABLE = True            # set False to skip crash join
CRASH_BUFFER_METERS = 10
OUTPUT_GEOJSON = "segments_lts_mvp.geojson"

OUT_DIR = "."
print("Output ->", os.path.join(OUT_DIR, OUTPUT_GEOJSON))

Output -> ./segments_lts_mvp.geojson


In [15]:
# 3) AOI polygon

aoi_gdf = ox.geocode_to_gdf(PLACE_NAME).to_crs(CRS)
aoi = aoi_gdf.geometry.iloc[0]
aoi_gdf

,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((-77.11979 38.93435, -77.11977 38.934...",-77.119795,38.79163,-76.909366,38.995968,394370670,relation,5396194,38.895037,-77.036543,place,city,16,0.803067,city,Washington,"Washington, District of Columbia, United States"


In [16]:
# 10) RideScore v1 (0–100)

def p95(values):
    s = sorted(v for v in values if pd.notnull(v))
    if not s: return 1
    k = int(round(0.95 * (len(s) - 1)))
    return max(int(s[k]), 1)

def lts_to_score(lts): return {1:100, 2:75, 3:40, 4:10}.get(int(lts), 10)

facility_bonus = {
    "protected_track": 10, "separated_lane": 10,
    "buffered_lane": 5, "painted_lane": 3,
    "shared": 0, "none": 0,
}

# Ensure gdf exists (create an empty placeholder if upstream cells haven't been run)
if 'gdf' not in globals():
    # minimal columns used by this cell; create empty GeoDataFrame so operations below won't error
    gdf = gpd.GeoDataFrame({
        'crash_count_5yr': pd.Series(dtype='int'),
        'lts_level': pd.Series(dtype='int'),
        'bike_facility_type': pd.Series(dtype='object'),
    }, geometry=pd.Series(dtype='geometry'), crs=CRS)
    print("Warning: 'gdf' was not defined — created empty GeoDataFrame placeholder. Run upstream cells to populate real data.")

P95_CRASH = p95(gdf["crash_count_5yr"])
def crash_inv_score(n): return 100.0 * (1.0 - min(max(float(n)/float(P95_CRASH), 0.0), 1.0))

W_LTS, W_CRASH, W_FAC = 0.6, 0.3, 0.1

gdf["s_LTS"] = gdf["lts_level"].map(lts_to_score)
gdf["s_crash"] = gdf["crash_count_5yr"].map(crash_inv_score)
gdf["s_facility"] = gdf["bike_facility_type"].map(facility_bonus).fillna(0)

gdf["ridescore_v1"] = (W_LTS*gdf.s_LTS + W_CRASH*gdf.s_crash + W_FAC*gdf.s_facility).round(1)
gdf[["s_LTS","s_crash","s_facility","ridescore_v1"]].describe().round(1)

# 11) Export GeoJSON

keep = ["segment_id","highway","num_lanes","speed_limit","bike_facility_type","parking_presence",
        "lts_level","crash_count_5yr","serious_injury_count_5yr","fatal_count_5yr","ridescore_v1","geometry"]

# Ensure required columns exist (with sensible defaults) to avoid KeyError during selection
defaults = {
    "segment_id": "",
    "highway": "",
    "num_lanes": 1,
    "speed_limit": 25,
    "bike_facility_type": "unknown",
    "parking_presence": False,
    "lts_level": 4,
    "crash_count_5yr": 0,
    "serious_injury_count_5yr": 0,
    "fatal_count_5yr": 0,
    "ridescore_v1": 10.0,
}
for c, d in defaults.items():
    if c not in gdf.columns:
        gdf[c] = d

# Ensure geometry exists and is set
if "geometry" not in gdf.columns:
    raise ValueError("gdf is missing 'geometry' column; run upstream cells to populate geometries")
gdf = gdf.set_geometry("geometry")

out_path = os.path.join(OUT_DIR, OUTPUT_GEOJSON)
gdf[keep].to_file(out_path, driver="GeoJSON")
print("Wrote:", out_path)

Wrote: ./segments_lts_mvp.geojson


In [17]:
# 12) Folium map (3 toggles + title) — DIAGNOSTICS + SAVE
import folium
from folium.plugins import MeasureControl
from branca.element import Element
import json, time, os

from matplotlib.pyplot import gcf

# Helper function to extract coordinates from geometry
def extract_coords(geom):
    """Extract coordinate list from a geometry object.

    Handles LineString, LinearRing, MultiLineString, Point and other
    iterable geometries. Returns a flat list of coordinate tuples.
    """
    if geom is None:
        return []
    # empty geometries
    try:
        if geom.is_empty:
            return []
    except Exception:
        pass

    geom_type = getattr(geom, "geom_type", None)
    # LineString / LinearRing: direct coords
    if geom_type in ("LineString", "LinearRing"):
        return list(geom.coords)
    # MultiLineString or GeometryCollection: concatenate parts that have coords
    if geom_type in ("MultiLineString", "GeometryCollection"):
        coords = []
        for part in geom:
            try:
                if getattr(part, "is_empty", False):
                    continue
                coords.extend(list(part.coords))
            except Exception:
                # skip parts that don't expose coords
                continue
        return coords
    # Point: return single coordinate
    if geom_type == "Point":
        try:
            return [tuple(geom.coords)[0]]
        except Exception:
            return []
    # Fallback: try to use .coords or iterate
    try:
        return list(geom.coords)
    except Exception:
        try:
            coords = []
            for part in geom:
                try:
                    coords.extend(list(part.coords))
                except Exception:
                    continue
            return coords
        except Exception:
            return []

# diagnostics: sizes and timing
print("Cells running: Folium map diagnostics")
try:
    n_segs = len(gdf)
except Exception as e:
    print("ERROR: `gdf` is not defined in the kernel. Run earlier cells first.")
    raise
print(f"Segments in gdf: {n_segs}")

start = time.time()
print("Extracting coordinates from geometries...")
gdf['coords'] = gdf.geometry.apply(extract_coords)
print(f"coords column created in {time.time() - start:.2f}s")

# quick inspect first 5
print("Sample coords lengths (first 5 rows):", [len(c) for c in gdf['coords'].head(5).tolist()])
if n_segs == 0:
    print("gdf empty — nothing to render")

# build a small sample (200 or fewer) to validate rendering quickly
sample_n = min(200, n_segs)
print(f"Building a small sample GeoJSON of {sample_n} features for inline display...")
start = time.time()
idxs = gdf.sample(n=sample_n, random_state=1).index
features_sample = []
for i in idxs:
    row = gdf.loc[i]
    coords = row['coords']
    features_sample.append({
        "type": "Feature",
        "properties": {
            "segment_id": str(row.get('segment_id','')),
            "num_lanes": int(row.get('num_lanes',1) or 1),
            "speed_limit": int(row.get('speed_limit',25) or 25),
            "bike_facility_type": str(row.get('bike_facility_type','unknown') or 'unknown'),
            "parking_presence": bool(row.get('parking_presence', False)),
            "lts_level": int(row.get('lts_level',4) or 4),
            "crash_count_5yr": int(row.get('crash_count_5yr',0) or 0),
            "ridescore_v1": float(row.get('ridescore_v1',10.0) or 10.0),
        },
        "geometry": {"type":"LineString", "coordinates": [[float(x), float(y)] for x,y in coords]}
    })
geo_sample = {"type":"FeatureCollection", "features": features_sample}
print(f"Sample GeoJSON built in {time.time()-start:.2f}s")

# display the sample inline to confirm rendering (fast)
print("Rendering sample map inline (should show quickly)...")
ms = folium.Map(location=[38.9072, -77.0369], zoom_start=12, tiles="OpenStreetMap")
# Attach tooltip only if the requested fields exist in the sample properties
requested_fields = ["segment_id", "ridescore_v1", "lts_level", "bike_facility_type"]
tooltip_fields = []
if features_sample:
    first_props = features_sample[0].get('properties', {}) if isinstance(features_sample[0], dict) else {}
    tooltip_fields = [f for f in requested_fields if f in first_props]

tooltip = folium.GeoJsonTooltip(fields=tooltip_fields) if tooltip_fields else None
if tooltip is not None:
    folium.GeoJson(geo_sample, style_function=lambda f: {"color": "#3182bd", "weight": 2}, tooltip=tooltip).add_to(ms)
else:
    # no tooltip fields available — add layer without tooltip
    folium.GeoJson(geo_sample, style_function=lambda f: {"color": "#3182bd", "weight": 2}).add_to(ms)
folium.LayerControl(collapsed=False).add_to(ms)
ms.add_child(MeasureControl(primary_length_unit='meters', primary_area_unit='sqmeters'))
print("Displaying sample map now...")
display(ms)
print("Sample displayed — if this is visible, rendering works; next step: save full map HTML")

# Build full GeoJSON features but do not rely on notebook renderer to display — save to file instead
print("Building full GeoJSON (this may take some time depending on size)...")
start = time.time()
# fill defaults once
gdf_filled = gdf.copy()
# Safely ensure columns exist with sensible defaults. Using DataFrame.get can
# return a scalar when the column is missing, so avoid calling .fillna() on scalars.
defaults = {
    'segment_id': ('', str),
    'num_lanes': (1, int),
    'speed_limit': (25, int),
    'bike_facility_type': ('unknown', str),
    'parking_presence': (False, bool),
    'lts_level': (4, int),
    'crash_count_5yr': (0, int),
    'ridescore_v1': (10.0, float),
}
for col, (default, dtype) in defaults.items():
    if col in gdf_filled.columns:
        try:
            gdf_filled[col] = gdf_filled[col].fillna(default)
        except Exception:
            # if the column exists but doesn't support fillna, coerce by replacement
            gdf_filled.loc[gdf_filled[col].isna(), col] = default
    else:
        # create a column with the default value for every row
        gdf_filled[col] = pd.Series([default] * len(gdf_filled), index=gdf_filled.index)
    # coerce dtype when possible
    try:
        gdf_filled[col] = gdf_filled[col].astype(dtype)
    except Exception:
        pass

# Create GeoJSON directly from GeoDataFrame using to_file (faster than manual dict building)
out_geojson = os.path.join(OUT_DIR, 'segments_lts_full.geojson')
try:
    # select minimal columns to reduce size
    save_cols = ['segment_id','num_lanes','speed_limit','bike_facility_type','parking_presence','lts_level','crash_count_5yr','ridescore_v1','geometry']
    gdf_filled[save_cols].to_file(out_geojson, driver='GeoJSON')
    print(f"Saved full GeoJSON to: {out_geojson} (in {time.time()-start:.2f}s)")
except Exception as e:
    print("Failed to write GeoJSON via to_file:", e)
    # fallback: build feature collection in memory (may be slow)
    features = []
    for _, row in gdf_filled.iterrows():
        coords = row['coords']
        features.append({
            "type": "Feature",
            "properties": {
                "segment_id": str(row['segment_id']),
                "num_lanes": int(row['num_lanes']),
                "speed_limit": int(row['speed_limit']),
                "bike_facility_type": str(row['bike_facility_type']),
                "parking_presence": bool(row['parking_presence']),
                "lts_level": int(row['lts_level']),
                "crash_count_5yr": int(row['crash_count_5yr']),
                "ridescore_v1": float(row['ridescore_v1']),
            },
            "geometry": {"type":"LineString","coordinates":[[float(x),float(y)] for x,y in coords]}
        })
    geo = {"type":"FeatureCollection","features":features}
    with open(out_geojson, 'w') as fh:
        json.dump(geo, fh)
    print(f"Wrote fallback GeoJSON to {out_geojson} (in {time.time()-start:.2f}s)")

# Create full map and save HTML (do not rely on notebook to render full map)
print("Creating full interactive map and saving to HTML (browser recommended)...")
start = time.time()
m = folium.Map(location=[38.9072, -77.0369], zoom_start=12, tiles="OpenStreetMap")
# add GeoJSON by file (this avoids embedding huge Python dict into notebook output)
try:
    folium.GeoJson(out_geojson, name='segments').add_to(m)
except Exception as e:
    print("Error adding GeoJSON by file to map:", e)
    # as fallback add sample layer only
    folium.GeoJson(geo_sample, name='sample').add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.add_child(MeasureControl(primary_length_unit='meters', primary_area_unit='sqmeters'))

output_map_dir = os.path.join(OUT_DIR, 'maps')
os.makedirs(output_map_dir, exist_ok=True)
output_map_path = os.path.join(output_map_dir, 'segments_lts_full_map.html')
m.save(output_map_path)
print(f"Saved full interactive map to: {output_map_path} (save time: {time.time()-start:.2f}s)")

print("Diagnostics complete. -- Displaying small sample inline and saved full map to HTML.")
print("Open the saved HTML file in your browser if the notebook webview doesn't render the full map.")

# show small sample map again as final inline confirmation
display(ms)

Cells running: Folium map diagnostics
Segments in gdf: 0
Extracting coordinates from geometries...
coords column created in 0.00s
Sample coords lengths (first 5 rows): []
gdf empty — nothing to render
Building a small sample GeoJSON of 0 features for inline display...
Sample GeoJSON built in 0.00s
Rendering sample map inline (should show quickly)...
Displaying sample map now...


Sample displayed — if this is visible, rendering works; next step: save full map HTML
Building full GeoJSON (this may take some time depending on size)...
Saved full GeoJSON to: ./segments_lts_full.geojson (in 0.00s)
Creating full interactive map and saving to HTML (browser recommended)...
Saved full interactive map to: ./maps/segments_lts_full_map.html (save time: 0.01s)
Diagnostics complete. -- Displaying small sample inline and saved full map to HTML.
Open the saved HTML file in your browser if the notebook webview doesn't render the full map.
Saved full interactive map to: ./maps/segments_lts_full_map.html (save time: 0.01s)
Diagnostics complete. -- Displaying small sample inline and saved full map to HTML.
Open the saved HTML file in your browser if the notebook webview doesn't render the full map.


In [18]:
# TEST: Simple Folium map to diagnose display issue
import folium

print("Creating minimal test map...")
test_map = folium.Map(location=[38.9072, -77.0369], zoom_start=12)
print("Test map created successfully!")
print("Attempting to display...")
display(test_map)
print("Display call completed!")

Creating minimal test map...
Test map created successfully!
Attempting to display...


Display call completed!
